# Lasso Regression

- Its like the ridge regression, a **shrinkage** method, it differs in its choice of penalty:
- Lasso regression uses the $\mathcal{l}_1$ **penalty**, which adds the absolute value of the magnitude of coefficients to the cost function, on the parameters $\beta$. Meaning Lasso finds an assignment to $\beta$ that minimizes the function:
  $$
  f(\beta) = ||{X\beta - Y}||^{2}_{2} + \lambda ||\beta||_{1}
  $$
  where $\lambda$ is a hyperparameter and, as usual, $X$ is the training data and $Y$ the observations. The $\mathcal{l}_1$ penalty encourages **sparsity** in the learned parameters, and, as will be shown, can drive many coefficients to zero. In this sense, lasso is a continuous **feature selection** method.


In [1]:
import cvxpy as cp  # to fit a lasso model, to eval it, and how to tune the hyperparameter lambda
import numpy as np
import matplotlib.pyplot as plt

from quantflow.sp.poisson import PoissonProcess

from quantflow.utils import plot
import ipywidgets as widg
import plotly.graph_objects as go


In [2]:
def simulate():
    pr = PoissonProcess(intensity=intensity.value)
    paths = pr.sample(samples.value, 1, 1000)
    pdf = paths.pdf(delta=1)
    pdf["simulation"] = pdf["pdf"]
    pdf["analytical"] = pr.marginal(1).pdf(pdf.index)
    return pdf


def on_intensity_change(change):
    df = simulate()
    fig.data[0].x = df.index
    fig.data[0].y = df["simulation"]
    fig.data[1].x = df.index
    fig.data[1].y = df["analytical"]


intensity = widg.IntSlider(description="intensity")
samples = widg.IntSlider(description="paths", min=100, max=10_000, step=100)
intensity.value = 50
samples.value = 1000
intensity.observe(on_intensity_change)
samples.observe(on_intensity_change)

df = simulate()
simulation = go.Bar(x=df.index, y=df["simulation"], name="simulation")
analytical = go.Scatter(x=df.index, y=df["analytical"], name="analytical")
fig = go.FigureWidget(data=[simulation, analytical])

widg.VBox([intensity, samples, fig])


In [3]:
from quantflow.sp.heston import HestonJ
from quantflow.options.pricer import OptionPricer

pricer = OptionPricer(
    model=HestonJ.create(
        vol=0.5,
        kappa=2,
        rho=-0.3,
        sigma=0.8,
        theta=0.36,
        jump_fraction=0.3,
        jump_asymmetry=1.2,
    )
)

pricer


OptionPricer(model=HestonJ(variance_process=CIR(rate=0.175, kappa=2.0, sigma=0.8, theta=0.36, sample_algo=<SamplingAlgorithm.implicit: 'implicit'>), rho=-0.3, jumps_up=CompoundPoissonProcess[Exponential](intensity=100.0, jumps=Exponential(decay=67.21992649890744)), jumps_down=CompoundPoissonProcess[Exponential](intensity=100.0, jumps=Exponential(decay=80.66391179868893))), n=128, max_moneyness_ttm=1.5)

In [4]:
pricer.plot3d(max_moneyness_ttm=1.5, support=31).update_layout(
    height=1200, title="Heston Volatility Surface"
)


In [1]:
from optionvisualizer.visualizer import Visualizer
from financetoolkit import options as opt
from volvisualizer.volatility import Volatility

In [2]:
nvda_surf = Volatility(ticker = "NVDA", start_date = "2024-05-06", wait = 0.5, volume = 100)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:992)>